In [1]:

import panel as pn
import holoviews as hv
pn.extension('tabulator', 'plotly')
hv.extension('bokeh')
import hvplot.pandas

import plotly.graph_objects as go
from plotly.subplots import make_subplots

#import importlib
#import data_processing
#importlib.reload(data_processing)

from data_processing import *


import logging

import os
from pathlib import Path

In [2]:
quali_data = combine_csv_files('../data')
if quali_data is not None:
    quali_data = convert_time(quali_data)
    career_timeline_data = process_qualifying_data(quali_data)

INFO:data_processing:Processing qualifying data...
INFO:data_processing:Processing year: 2019


Successfully read: qualifying_data_2019_results.csv
Successfully read: qualifying_data_2022_results.csv
Successfully read: qualifying_data_2024_results.csv
Successfully read: qualifying_data_2021_results.csv
Successfully read: qualifying_data_2023_results.csv
Successfully read: qualifying_data_2018_results.csv
Successfully read: qualifying_data_2020_results.csv

Total number of files combined: 7
Total rows in DataFrame: 2819


INFO:data_processing:Processing year: 2022
INFO:data_processing:Processing year: 2024
INFO:data_processing:Processing year: 2021
INFO:data_processing:Processing year: 2023
INFO:data_processing:Processing year: 2018
INFO:data_processing:Processing year: 2020


In [3]:
career_timeline_data

[{'year': 2019,
  'driver': 'A GIOVINAZZI',
  'team': 'Alfa Romeo Racing',
  'events': [{'round': 'Abu Dhabi Grand Prix',
    'position': 17.0,
    'gapToPole': 3.335000000000008,
    'teammateGap': -0.26899999999999125,
    'hasTeammateData': True},
   {'round': 'Australian Grand Prix',
    'position': 14.0,
    'gapToPole': 2.2279999999999944,
    'teammateGap': 0.4000000000000057,
    'hasTeammateData': True},
   {'round': 'Austrian Grand Prix',
    'position': 8.0,
    'gapToPole': 1.176000000000002,
    'teammateGap': 0.01300000000000523,
    'hasTeammateData': True},
   {'round': 'Azerbaijan Grand Prix',
    'position': 8.0,
    'gapToPole': 1.929000000000002,
    'teammateGap': nan,
    'hasTeammateData': False},
   {'round': 'Bahrain Grand Prix',
    'position': 16.0,
    'gapToPole': 2.1599999999999966,
    'teammateGap': 1.0039999999999907,
    'hasTeammateData': True},
   {'round': 'Belgian Grand Prix',
    'position': 15.0,
    'gapToPole': 3.117999999999995,
    'teammateG

In [4]:
def combine_csv_files(folder_path):
    """
    Read all CSV files from a folder and combine them into a single DataFrame.
    
    Parameters:
    folder_path (str): Path to the folder containing CSV files
    
    Returns:
    pandas.DataFrame: Combined DataFrame from all CSV files
    """
    path = Path(folder_path)
    all_dfs = []
    
    # Loop through all files in the folder
    for file in path.glob('*.csv'):
        try:
            df = pd.read_csv(file)
            # Append to the list
            all_dfs.append(df)
            
            print(f"Successfully read: {file.name}")
            
        except Exception as e:
            print(f"Error reading {file.name}: {str(e)}")
    
    # Combine all DataFrames
    if all_dfs:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        print(f"\nTotal number of files combined: {len(all_dfs)}")
        print(f"Total rows in combined DataFrame: {len(combined_df)}")
        return combined_df
    else:
        print("No CSV files found in the specified folder!")
        return None

In [5]:
quali_data = combine_csv_files('../data')
print(quali_data['Year'].unique())
quali_data.head()

Successfully read: qualifying_data_2019_results.csv
Successfully read: qualifying_data_2022_results.csv
Successfully read: qualifying_data_2024_results.csv
Successfully read: qualifying_data_2021_results.csv
Successfully read: qualifying_data_2023_results.csv
Successfully read: qualifying_data_2018_results.csv
Successfully read: qualifying_data_2020_results.csv

Total number of files combined: 7
Total rows in combined DataFrame: 2819
[2019 2022 2024 2021 2023 2018 2020]


,DriverNumber,BroadcastName,TeamName,Position,Q1,Q2,Q3,Year,EventName,WetSession
0,44,L HAMILTON,Mercedes,1.0,0 days 00:01:22.043000,0 days 00:01:21.014000,0 days 00:01:20.486000,2019,Australian Grand Prix,False
1,77,V BOTTAS,Mercedes,2.0,0 days 00:01:22.367000,0 days 00:01:21.193000,0 days 00:01:20.598000,2019,Australian Grand Prix,False
2,5,S VETTEL,Ferrari,3.0,0 days 00:01:22.885000,0 days 00:01:21.912000,0 days 00:01:21.190000,2019,Australian Grand Prix,False
3,33,M VERSTAPPEN,Red Bull Racing,4.0,0 days 00:01:22.876000,0 days 00:01:21.678000,0 days 00:01:21.320000,2019,Australian Grand Prix,False
4,16,C LECLERC,Ferrari,5.0,0 days 00:01:22.017000,0 days 00:01:21.739000,0 days 00:01:21.442000,2019,Australian Grand Prix,False


In [6]:
print(quali_data[['Q1', 'Q2', 'Q3']].dtypes)

Q1    object
Q2    object
Q3    object
dtype: object


In [7]:
# Convert from string to Timedelta
quali_data['Q1'] = pd.to_timedelta(quali_data['Q1'])
quali_data['Q2'] = pd.to_timedelta(quali_data['Q2'])
quali_data['Q3'] = pd.to_timedelta(quali_data['Q3'])

# Now convert the Timedelta columns to total seconds
quali_data['Q1Seconds'] = quali_data['Q1'].apply(lambda x: x.total_seconds())
quali_data['Q2Seconds'] = quali_data['Q2'].apply(lambda x: x.total_seconds())
quali_data['Q3Seconds'] = quali_data['Q3'].apply(lambda x: x.total_seconds())

quali_data.head()

,DriverNumber,BroadcastName,TeamName,Position,Q1,Q2,Q3,Year,EventName,WetSession,Q1Seconds,Q2Seconds,Q3Seconds
0,44,L HAMILTON,Mercedes,1.0,0 days 00:01:22.043000,0 days 00:01:21.014000,0 days 00:01:20.486000,2019,Australian Grand Prix,False,82.043,81.014,80.486
1,77,V BOTTAS,Mercedes,2.0,0 days 00:01:22.367000,0 days 00:01:21.193000,0 days 00:01:20.598000,2019,Australian Grand Prix,False,82.367,81.193,80.598
2,5,S VETTEL,Ferrari,3.0,0 days 00:01:22.885000,0 days 00:01:21.912000,0 days 00:01:21.190000,2019,Australian Grand Prix,False,82.885,81.912,81.190
3,33,M VERSTAPPEN,Red Bull Racing,4.0,0 days 00:01:22.876000,0 days 00:01:21.678000,0 days 00:01:21.320000,2019,Australian Grand Prix,False,82.876,81.678,81.320
4,16,C LECLERC,Ferrari,5.0,0 days 00:01:22.017000,0 days 00:01:21.739000,0 days 00:01:21.442000,2019,Australian Grand Prix,False,82.017,81.739,81.442


In [8]:
quali_data.columns

Index(['DriverNumber', 'BroadcastName', 'TeamName', 'Position', 'Q1', 'Q2',
       'Q3', 'Year', 'EventName', 'WetSession', 'Q1Seconds', 'Q2Seconds',
       'Q3Seconds'],
      dtype='object')

In [9]:

from typing import List, Dict
import logging


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


def process_qualifying_data(quali_data: pd.DataFrame) -> List[Dict]:
    """
    Process F1 qualifying data .
    
    """
    timeline_data = []
    
    # Verify cols
    required_columns = ['DriverNumber', 'BroadcastName', 'TeamName', 'Position', 'Q1', 'Q2',
       'Q3', 'Year', 'EventName', 'WetSession', 'Q1Seconds', 'Q2Seconds',
       'Q3Seconds']
    missing_columns = [col for col in required_columns if col not in quali_data.columns]
    if missing_columns:
        raise ValueError(f"Missing required columns: {missing_columns}")
    
    for year in quali_data['Year'].unique():
        year_data = quali_data[quali_data['Year'] == year]
        
        for event_name, event_data in year_data.groupby('EventName'):
            #logger.info(f"Processing {event_name} {year}")
            
            # Get pole position data (P1)
            pole_data = event_data[event_data['Position'] == 1].iloc[0]
            #pole_driver = pole_data['BroadcastName']
            pole_time = pole_data['Q3Seconds']
            
            team_groups = event_data.groupby('TeamName')
            
            for team, team_data in team_groups:
                drivers = team_data['BroadcastName'].unique()
                
                # Initialize gaps dict
                gaps = {driver: np.nan for driver in drivers}
                
                # Calculate teammate gaps only if 2 drivers
                if len(drivers) == 2:
                    driver1, driver2 = drivers
                    time1_data = team_data[team_data['BroadcastName'] == driver1].iloc[0]
                    time2_data = team_data[team_data['BroadcastName'] == driver2].iloc[0]
                    
                    time1 = None
                    if pd.notna(time1_data['Q3Seconds']):
                        time1 = time1_data['Q3Seconds']
                    elif pd.notna(time1_data['Q2Seconds']):
                        time1 = time1_data['Q2Seconds']
                    elif pd.notna(time1_data['Q1Seconds']):
                        time1 = time1_data['Q1Seconds']

                        
                    time2 = None  
                    if pd.notna(time2_data['Q3Seconds']):
                        time2 = time2_data['Q3Seconds']
                    elif pd.notna(time2_data['Q2Seconds']):
                        time2 = time2_data['Q2Seconds']
                    elif pd.notna(time2_data['Q1Seconds']):
                        time2 = time2_data['Q1Seconds']

                    if time1 is not None and time2 is not None:
                        gaps.update({                  #check 
                            driver1: time1 - time2,
                            driver2: time2 - time1
                        })
                else:
                    #pass
                    logger.info(f'Incomplete teammate data: {len(drivers)} driver(s) for {team} at {event_name} {year}')
                
                # Process each driver
                for driver in drivers:
                    driver_data = team_data[team_data['BroadcastName'] == driver].iloc[0]
                    best_time = None

                    if pd.notna(driver_data['Q3Seconds']):
                        best_time = driver_data['Q3Seconds']
                    elif pd.notna(driver_data['Q2Seconds']):
                        best_time = driver_data['Q2Seconds']
                    elif pd.notna(driver_data['Q1Seconds']):
                        best_time = driver_data['Q1Seconds']

                    qualifying_position = driver_data['Position'] if pd.notna(driver_data['Position']) else np.nan

                    
                    # Calculate gap to pole
                    if pd.isna(qualifying_position):
                        gap_to_pole = np.nan
                    elif qualifying_position == 1:
                        gap_to_pole = 0.0
                    elif best_time is None or pd.isna(pole_time):
                        gap_to_pole = np.nan
                    else:
                        gap_to_pole = best_time - pole_time 
                    
                    # Create event summary
                    event_summary = {
                        'round': event_name,
                        'position': qualifying_position,
                        'gapToPole': gap_to_pole,
                        'teammateGap': gaps[driver],
                        'hasTeammateData': not pd.isna(gaps[driver])
                    }
                    
                    # Find or create driver entry for this year
                    driver_entry = next(
                        (item for item in timeline_data if item['year'] == year and item['driver'] == driver),
                        None
                    )
                    
                    if driver_entry is None:
                        driver_entry = {
                            'year': year,
                            'driver': driver,
                            'team': team,
                            'events': [],
                            'positions': [],
                            'gapToPole_values': [],
                            'teammateGap_values': [],
                            'completeDataCount': 0,
                            'totalEvents': 0
                        }
                        timeline_data.append(driver_entry)
                    
                    driver_entry['events'].append(event_summary)
                    driver_entry['positions'].append(qualifying_position)
                    driver_entry['totalEvents'] += 1
                    
                    if gap_to_pole is not None and gap_to_pole != 'N/A':  #need to remove N/A options from this code 
                        driver_entry['gapToPole_values'].append(gap_to_pole)
                    if gaps[driver] is not None and gaps[driver] != 'N/A':
                        driver_entry['teammateGap_values'].append(gaps[driver])
                        driver_entry['completeDataCount'] += 1
    
    # Calculate final averages
    for entry in timeline_data:
        # Only include positions that are not NaN
        valid_positions = [pos for pos in entry['positions'] if not pd.isna(pos)]
        entry['avgQualifyingPosition'] = np.mean(valid_positions) if valid_positions else np.nan
    
         #Only include valid gap to pole times (exclude 'N/A' and NaN)
        valid_gaps_to_pole = [gap for gap in entry['gapToPole_values'] if not pd.isna(gap) and gap != 'N/A']
        entry['avgGapToPole'] = np.mean(valid_gaps_to_pole) if valid_gaps_to_pole else np.nan
    
        # Only include teammate gaps where there was actually a comparison
        valid_teammate_gaps = [gap for gap in entry['teammateGap_values'] if not pd.isna(gap) and gap != 'N/A']
        entry['avgTeammateGap'] = np.mean(valid_teammate_gaps) if valid_teammate_gaps else np.nan
    
        entry['dataCompleteness'] = entry['completeDataCount'] / entry['totalEvents'] if entry['totalEvents'] > 0 else 0
        
        del entry['positions']
        del entry['gapToPole_values']
        del entry['teammateGap_values']
        del entry['completeDataCount']
        del entry['totalEvents']
    
    return timeline_data

In [10]:
career_timeline_data = process_qualifying_data(quali_data)


INFO:__main__:Incomplete teammate data: 1 driver(s) for Williams at Australian Grand Prix 2024


In [11]:
career_timeline_data[:5]

[{'year': 2019,
  'driver': 'A GIOVINAZZI',
  'team': 'Alfa Romeo Racing',
  'events': [{'round': 'Abu Dhabi Grand Prix',
    'position': 17.0,
    'gapToPole': 3.335000000000008,
    'teammateGap': -0.26899999999999125,
    'hasTeammateData': True},
   {'round': 'Australian Grand Prix',
    'position': 14.0,
    'gapToPole': 2.2279999999999944,
    'teammateGap': 0.4000000000000057,
    'hasTeammateData': True},
   {'round': 'Austrian Grand Prix',
    'position': 8.0,
    'gapToPole': 1.176000000000002,
    'teammateGap': 0.01300000000000523,
    'hasTeammateData': True},
   {'round': 'Azerbaijan Grand Prix',
    'position': 8.0,
    'gapToPole': 1.929000000000002,
    'teammateGap': nan,
    'hasTeammateData': False},
   {'round': 'Bahrain Grand Prix',
    'position': 16.0,
    'gapToPole': 2.1599999999999966,
    'teammateGap': 1.0039999999999907,
    'hasTeammateData': True},
   {'round': 'Belgian Grand Prix',
    'position': 15.0,
    'gapToPole': 3.117999999999995,
    'teammateG

### Dashboard 


In [44]:
from panel.template import DarkTheme
import panel as pn
import pandas as pd

# Set up theme and styling
pn.extension(sizing_mode="stretch_width")
pn.config.sizing_mode = "stretch_width"

def create_driver_timeline(timeline_data):
    df = pd.DataFrame(timeline_data)
    all_drivers = sorted(df['driver'].unique().tolist())
    
    driver_selector = pn.widgets.Select(
        name='Select Driver',
        options=all_drivers,
        value=all_drivers[0],  #initial value
        width=300,
        styles={
            'background': '#f8f9fa',
            'border': '1px solid #dee2e6',
            'border-radius': '8px',
            'padding': '8px',
            'font-family': 'Inter, sans-serif'
        }
    )
    
    def create_timeline(driver):
        if not driver:
            return pn.Column(
                pn.pane.Markdown("Please select a driver", styles={'font-family': 'Inter, sans-serif'})
            )
        
        def create_year_panel(year_data):
            events_df = pd.DataFrame(year_data['events'])
            valid_pos_df = events_df.dropna(subset=['position'])
            
            year_marker = pn.pane.Markdown(
                f"## {year_data['year']}", 
                styles={
                    'margin-bottom': '15px',
                    'padding': '15px',
                    'font-size': '28px',
                    'font-weight': '600',
                    'font-family': 'Inter, sans-serif',
                    'color': '#212529',
                    'cursor': 'default'
                }
            )
            
            scatter_plot = valid_pos_df.hvplot.scatter(
                'round', 'position',
                size=25,
                color='#dc3545',
                hover_cols=['round', 'gapToPole', 'teammateGap']
            ).opts(
                padding=0.1,
                show_grid=True,
                fontsize={'labels': 14, 'xticks': 12, 'yticks': 12, 'title': 20},
                xrotation=45,
                margin=(50,50,100,50),
                tools=['hover', 'box_zoom', 'reset'],
                xlabel='Race',
                ylabel='Position',
                width=1500,
                height=400,
                bgcolor='#f8f9fa',
                axiswise=True
            )
            
            best_position = valid_pos_df['position'].min()
            best_races = valid_pos_df[valid_pos_df['position'] == best_position]['round'].tolist()
            
            header = pn.Column(
                pn.Row(
                    pn.pane.Markdown(
                        f"🏆 **Best Position:** P{best_position:.0f}",
                        styles={'font-size': '18px', 'background': '#fff', 'padding': '10px', 'border-radius': '8px', 'border': '1px solid #dee2e6'}
                    ),
                    pn.pane.Markdown(
                        f"📅 **P{best_position:.0f} Achieved at:** {', '.join(best_races)}",
                        styles={'font-size': '18px', 'background': '#fff', 'padding': '10px', 'border-radius': '8px', 'border': '1px solid #dee2e6'}
                    ),
                    styles={'gap': '20px'}
                )
            )

            metrics = pn.Row(
                pn.pane.Markdown(f"🏎 **Team:** {year_data['team']}", styles={'font-size': '16px', 'background': '#fff', 'padding': '10px', 'border-radius': '8px', 'border': '1px solid #dee2e6'}),
                pn.pane.Markdown(f"📊 **Avg Qualifying Position:** {year_data['avgQualifyingPosition']:.2f}", styles={'font-size': '16px', 'background': '#fff', 'padding': '10px', 'border-radius': '8px', 'border': '1px solid #dee2e6'}),
                pn.pane.Markdown(f"⏱ **Avg Gap to Pole:** {year_data['avgGapToPole']:.3f}s", styles={'font-size': '16px', 'background': '#fff', 'padding': '10px', 'border-radius': '8px', 'border': '1px solid #dee2e6'}),
                pn.pane.Markdown(f"🔄 **Avg Gap to Teammate:** {year_data['avgTeammateGap']:.3f}s", styles={'font-size': '16px', 'background': '#fff', 'padding': '10px', 'border-radius': '8px', 'border': '1px solid #dee2e6'}),
                styles={'gap': '20px'}
            )
            
            
            race_selector = pn.widgets.Select(
                name='Select Race',
                options=valid_pos_df['round'].tolist(),
                value=valid_pos_df['round'].iloc[0],  #Set initial value
                width=300,
                styles={
                    'background': '#f8f9fa',
                    'border': '1px solid #dee2e6',
                    'border-radius': '8px',
                    'padding': '8px',
                    'font-family': 'Inter, sans-serif'
                }
            )


            race_details = pn.Row(styles={'gap': '20px'})

            def update_race_details(event):
                race_data = valid_pos_df[valid_pos_df['round'] == race_selector.value].iloc[0]
                
                position = f"P{race_data['position']:.0f}"
                pole_gap = f"+{race_data['gapToPole']:.3f}s" if race_data['gapToPole'] > 0 else "0.000s"
                teammate_gap = f"{race_data['teammateGap']:+.3f}s" if race_data['hasTeammateData'] else "No Teammate Data"
                
                race_details.clear()
                race_details.extend([
                    pn.pane.Markdown(
                        f"🏁 **{driver}'s Qualifying Position:** {position}",
                        styles={'font-size': '16px', 'background': '#fff', 'padding': '10px', 'border-radius': '8px', 'border': '1px solid #dee2e6'}
                    ),
                    pn.pane.Markdown(
                        f"⏱ **Gap to Pole:** {pole_gap}",
                        styles={'font-size': '16px', 'background': '#fff', 'padding': '10px', 'border-radius': '8px', 'border': '1px solid #dee2e6'}
                    ),
                    pn.pane.Markdown(
                        f"🔄 **Gap to Teammate:** {teammate_gap}",
                        styles={'font-size': '16px', 'background': '#fff', 'padding': '10px', 'border-radius': '8px', 'border': '1px solid #dee2e6'}
                    )
                ])
            
            # Initialize race details
            update_race_details(None)
            race_selector.param.watch(update_race_details, 'value')
            
            return pn.Column(
                pn.Column(
                    year_marker,
                    styles={
                        'margin-bottom': '10px'  
                    },
                    sizing_mode='stretch_width'
                ),
                pn.Column(
                    header,
                    metrics,
                    pn.pane.HoloViews(scatter_plot, margin=(20, 0)),
                    pn.Row(
                        pn.pane.Markdown(
                            f"**Showing Specific Event Details For the {race_selector.value}**", #THIS ISNT RIGHT - FIX 
                            styles={
                                'font-size': '22px',
                                'font-family': 'Inter, sans-serif',
                                'color': '#212529',
                                'margin': '0',  #Remove margin 
                                'align-self': 'center'  #Center vertically 
                            }
                        ),
                        race_selector,
                        styles={
                            'gap': '5px',  
                            'align-items': 'center',  #Center vertically
                            'margin': '20px 0'  
                        }
                    ),
                    race_details,
                    styles={
                        'background': '#ffffff',
                        'padding': '20px',
                        'border-radius': '12px',
                        'border': '1px solid #dee2e6',
                        'box-shadow': '0 2px 4px rgba(0,0,0,0.1)',
                        'margin-top': '10px',
                    },
                    sizing_mode='stretch_width'
                ),
                pn.layout.Divider(margin=(30, 0)),
                sizing_mode='stretch_width'
            )
        
        driver_data = df[df['driver'] == driver].sort_values('year')
        timeline_panels = [create_year_panel(year_data) for _, year_data in driver_data.iterrows()]
        
        return pn.Column(
            pn.pane.Markdown(
                f"# {driver}'s Performance Timeline",
                styles={
                    'font-size': '36px',
                    'font-weight': '700',
                    'font-family': 'Inter, sans-serif',
                    'margin-bottom': '30px',
                    'color': '#212529',
                    'text-align': 'center',
                    'cursor': 'default'
                }
            ),
            *timeline_panels,
            styles={'gap': '30px'}
        )
    
    
    dynamic_panel = pn.bind(create_timeline, driver_selector)
    
    return pn.Column(
        pn.Row(
            driver_selector,
            styles={'justify-content': 'center', 'margin': '20px 0'}
        ),
        pn.layout.Divider(),
        dynamic_panel,
        styles={
            'background': '#f8f9fa',
            'padding': '20px'
        }
    )

In [45]:
dashboard = create_driver_timeline(career_timeline_data)

In [46]:
dashboard.show()


INFO:bokeh.server.server:Starting Bokeh server version 3.6.0 (running on Tornado 6.4.1)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


Launching server at http://localhost:59820


In [18]:

dashboard.servable()

Column(sizing_mode='stretch_width', styles={'background': '#f8f9fa', ...})
    [0] Row(sizing_mode='stretch_width', styles={'justify-content': 'cente...})
        [0] Select(options=['A ALBON', 'A GIOVINAZZI'...], styles={'background': '#f8f9fa', ...}, value='A ALBON')
    [1] Divider(sizing_mode='stretch_width')
    [2] Column(sizing_mode='stretch_width', styles={'gap': '30px'})
        [0] Markdown(str, sizing_mode='stretch_width', styles={'font-size': '36px', ...})
        [1] Column(sizing_mode='stretch_width')
            [0] Column(sizing_mode='stretch_width', styles={'margin-bottom': '10px'})
                [0] Markdown(str, sizing_mode='stretch_width', styles={'margin-bottom': '15px', ...})
            [1] Column(sizing_mode='stretch_width', styles={'background': '#ffffff', ...})
                [0] Column(sizing_mode='stretch_width')
                    [0] Row(sizing_mode='stretch_width', styles={'gap': '20px'})
                        [0] Markdown(str, sizing_mode='stretch_width', styles={'font-size': '18px', ...})
                        [1] Markdown(str, sizing_mode='stretch_width', styles={'font-size': '18px', ...})
                [1] Row(sizing_mode='stretch_width', styles={'gap': '20px'})
                    [0] Markdown(str, sizing_mode='stretch_width', styles={'font-size': '16px', ...})
                    [1] Markdown(str, sizing_mode='stretch_width', styles={'font-size': '16px', ...})
                    [2] Markdown(str, sizing_mode='stretch_width', styles={'font-size': '16px', ...})
                    [3] Markdown(str, sizing_mode='stretch_width', styles={'font-size': '16px', ...})
                [2] HoloViews(Scatter, height=400, margin=(20, 0), sizing_mode='fixed', width=1500)
                [3] Markdown(str, sizing_mode='stretch_width', styles={'font-size': '22px', ...})
                [4] Row(sizing_mode='stretch_width')
                    [0] Select(options=['Abu Dhabi Grand Prix', ...], styles={'background': '#f8f9fa', ...}, value='Abu Dhabi Grand Prix')
                [5] Row(sizing_mode='stretch_width', styles={'gap': '20px'})
                    [0] Markdown(str, sizing_mode='stretch_width', styles={'font-size': '16px', ...})
                    [1] Markdown(str, sizing_mode='stretch_width', styles={'font-size': '16px', ...})
                    [2] Markdown(str, sizing_mode='stretch_width', styles={'font-size': '16px', ...})
            [2] Divider(margin=(30, 0), sizing_mode='stretch_width')
        [2] Column(sizing_mode='stretch_width')
            [0] Column(sizing_mode='stretch_width', styles={'margin-bottom': '10px'})
                [0] Markdown(str, sizing_mode='stretch_width', styles={'margin-bottom': '15px', ...})
            [1] Column(sizing_mode='stretch_width', styles={'background': '#ffffff', ...})
                [0] Column(sizing_mode='stretch_width')
                    [0] Row(sizing_mode='stretch_width', styles={'gap': '20px'})
                        [0] Markdown(str, sizing_mode='stretch_width', styles={'font-size': '18px', ...})
                        [1] Markdown(str, sizing_mode='stretch_width', styles={'font-size': '18px', ...})
                [1] Row(sizing_mode='stretch_width', styles={'gap': '20px'})
                    [0] Markdown(str, sizing_mode='stretch_width', styles={'font-size': '16px', ...})
                    [1] Markdown(str, sizing_mode='stretch_width', styles={'font-size': '16px', ...})
                    [2] Markdown(str, sizing_mode='stretch_width', styles={'font-size': '16px', ...})
                    [3] Markdown(str, sizing_mode='stretch_width', styles={'font-size': '16px', ...})
                [2] HoloViews(Scatter, height=400, margin=(20, 0), sizing_mode='fixed', width=1500)
                [3] Markdown(str, sizing_mode='stretch_width', styles={'font-size': '22px', ...})
                [4] Row(sizing_mode='stretch_width')
                    [0] Select(options=['70th Anniversary Grand P...], styles={'background': '#f8f9fa', ...}, va